Use pydocstyle to check docstrings in the stubs and callect and export the results to a json file.
This json can then be imported in PowerBI to analyze theresults and changes over time.

1. Run this notebook `(Docstrings_check.ipynb)`

   - this will run pydocstyle on the stubs and export the results to a json file (`docstrings.json`)

2. Open the PowerBI file `(Docstring_View_.pbix)`
   - Refresh the data
   - Analyze the results


In [11]:
# !pydocstyle --match='(?!test_).*\.py(i)?' .
# !pydocstyle --match='(?!test_).*\.py(i)?' --add-ignore=D105,D107,  ../repos/micropython-stubs/publish/micropython-v1_21_0-esp32-stubs

# messages = !pydocstyle --match='(?!test_).*\.py(i)?' --add-ignore=D2,D4 --select=D100,D101,D102,D103,D104  ../repos/micropython-stubs/publish/micropython-v1_21_0-esp32-stubs
# print(messages[1:4])

In [12]:
from pathlib import Path


def count_def_class(folder_path: Path) -> dict:
    folder_path = Path(folder_path)
    if not folder_path.is_dir():
        raise ValueError(f"{folder_path} is not a directory")
    counts = {"def": 0, "class": 0, "module": 0}
    for file_path in folder_path.rglob("*"):
        if file_path.is_file() and file_path.suffix in [".pyi", ".py"]:
            counts["module"] += 1
            with file_path.open(encoding="utf8") as f:
                content = f.read()
                counts["def"] += content.count("def ")
                counts["class"] += content.count("class ")
    return counts


# count_def_class("../repos/micropython-stubs/publish/micropython-v1_21_0-esp32-stubs")

In [13]:
import pydocstyle

from functools import lru_cache


@lru_cache(maxsize=1000)
def count_missing_docstrings(folder_path: Path) -> dict:
    folder_path = Path(folder_path)
    if not folder_path.is_dir():
        raise ValueError(f"{folder_path} is not a directory")
    files = []
    for file_path in folder_path.rglob("*"):
        if (
            file_path.is_file()
            and file_path.suffix in [".pyi", ".py"]
            and not "stdlib" in str(file_path)
        ):
            files.append(str(file_path))

    # parser = pydocstyle.config.ConfigurationParser()

    docstring_checks = "D100,D101,D102,D103,D104"
    try:
        errors = pydocstyle.check(files, select=docstring_checks)
        messages = list(
            f"{e.code} {e.definition} in {e.filename}"
            for e in errors
            if isinstance(e, pydocstyle.Error)
        )
    except Exception as e:
        messages = []
    # print(messages[1:4])
    missing = {
        "module": len([m for m in messages if "D100" in m]),  # module or package
        "class": len([m for m in messages if "D101" in m]),  # class
        "def": len([m for m in messages if "D102" in m or "D103" in m]),  # method or function
    }
    return missing


# pth = Path("../repos/micropython-stubs/publish/micropython-v1_21_0-esp32-stubs")
# missing = count_missing_docstrings(pth)
# print(missing)

In [14]:
def get_version_port(folder_name: str) -> tuple:
    version = "unknown"
    port = "unknown"
    try:
        version = folder_name.split("-")[1]
        port = folder_name.split("-")[2]
    except IndexError:
        pass
    if version == "stdlib":
        port = "stdlib"
        version = "1.0"
    version = version.replace("_", ".").lstrip("v")
    if version == "latest":
        version = "99.99.99"
    return version, port


def get_docstring_score(folder_path: Path) -> dict:
    defs = count_def_class(folder_path)
    missing = count_missing_docstrings(folder_path)
    docstring_score = {
        "folder": folder_path.name,
        "path": str(folder_path),
        "version": get_version_port(folder_path.name)[0],
        "port": get_version_port(folder_path.name)[1],
        "module": round(1 - (missing["module"]) / (defs["module"]), 2) if defs["module"] else 1.0,
        "class": round(1 - (missing["class"]) / (defs["class"]), 2) if defs["class"] else 1.0,
        "def": round(1 - (missing["def"]) / (defs["def"]), 2) if defs["def"] else 1.0,
        "counts": defs,
        "missing": missing,
    }
    return docstring_score


def get_scores(publish_path: Path, *, match="micropython-*") -> list:
    scores = []
    for folder_path in publish_path.glob(match):
        if not folder_path.is_dir():
            continue
        docstring_score = get_docstring_score(folder_path)
        scores.append(docstring_score)
    return scores


scores = []
scores += get_scores(Path("../repos/micropython-stubs/stubs", match="*-merged"))
scores += get_scores(Path("../repos/micropython-stubs/publish"))

print(len(scores))

Error in file ..\repos\micropython-stubs\stubs\micropython-latest-frozen\esp32\ARDUINO_NANO_ESP32\aioespnow.py: Cannot parse file.
Error in file ..\repos\micropython-stubs\stubs\micropython-latest-frozen\esp32\GENERIC\aioespnow.py: Cannot parse file.
Error in file ..\repos\micropython-stubs\stubs\micropython-latest-frozen\esp32\LILYGO_TTGO_LORA32\aioespnow.py: Cannot parse file.
Error in file ..\repos\micropython-stubs\stubs\micropython-latest-frozen\esp32\LOLIN_C3_MINI\aioespnow.py: Cannot parse file.
Error in file ..\repos\micropython-stubs\stubs\micropython-latest-frozen\esp32\LOLIN_S2_MINI\aioespnow.py: Cannot parse file.
Error in file ..\repos\micropython-stubs\stubs\micropython-latest-frozen\esp32\LOLIN_S2_PICO\aioespnow.py: Cannot parse file.
Error in file ..\repos\micropython-stubs\stubs\micropython-latest-frozen\esp32\M5STACK_ATOM\aioespnow.py: Cannot parse file.
Error in file ..\repos\micropython-stubs\stubs\micropython-latest-frozen\esp32\UM_FEATHERS2\aioespnow.py: Cannot pa

197


In [15]:
import json

with open("docstrings.json", "w", encoding="utf-8") as file:
    json.dump(scores, file, ensure_ascii=False, indent=4)


# write to csv
import csv

with open("docstrings.csv", "w", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=scores[0].keys())
    writer.writeheader()
    writer.writerows(scores)

In [16]:
# %pip install chime

# importing chime package
import chime

# Successfully running the code sounds
chime.theme("big-sur")
chime.success()

In [17]:
import pandas as pd

# import numpy as np

df = pd.DataFrame(scores)

In [18]:
df.head()

,folder,path,version,port,module,class,def,counts,missing
0,micropython-core,..\repos\micropython-stubs\stubs\micropython-core,core,unknown,1.00,1.00,1.00,"{'def': 14, 'class': 1, 'module': 2}","{'module': 0, 'class': 0, 'def': 0}"
1,micropython-latest-docstubs,..\repos\micropython-stubs\stubs\micropython-l...,99.99.99,docstubs,1.00,0.96,0.97,"{'def': 919, 'class': 120, 'module': 55}","{'module': 0, 'class': 5, 'def': 24}"
2,micropython-latest-esp32-ESP32_GENERIC-merged,..\repos\micropython-stubs\stubs\micropython-l...,99.99.99,esp32,0.84,0.71,0.59,"{'def': 2248, 'class': 385, 'module': 174}","{'module': 28, 'class': 110, 'def': 918}"
3,micropython-latest-esp32-merged,..\repos\micropython-stubs\stubs\micropython-l...,99.99.99,esp32,0.83,0.70,0.58,"{'def': 2236, 'class': 377, 'module': 174}","{'module': 30, 'class': 112, 'def': 950}"
4,micropython-latest-esp8266-merged,..\repos\micropython-stubs\stubs\micropython-l...,99.99.99,esp8266,0.89,0.80,0.62,"{'def': 1322, 'class': 166, 'module': 116}","{'module': 13, 'class': 34, 'def': 498}"
